In [ ]:
from scipy.sparse import csr_matrix
import tables
import h5py
import hdf5plugin

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np
import seaborn as sb
from scipy import signal
import glob

import pandas as pd

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 200

# Import contact frequency matrices


In [ ]:
# import find raw data files

file_names = []
for name in glob.glob('./Raw data/*_5kb.h5'):
    file_names.append(name)

# Define functions useful for analysis

In [ ]:
# import HiC heat map data as a matrix

#argument:
# filename: input name of the file you wish to import matrix for

#output:
#the heat map itself as a matrix

def make_matrix(filename):
    with tables.open_file(filename, 'r') as f:
        parts = {}
        try:
            for matrix_part in ('data', 'indices', 'indptr', 'shape'):
                parts[matrix_part] = getattr(f.root.matrix, matrix_part).read()
        except Exception as e:
            log.info('No h5 file. Please check parameters concerning the file type!')
            e
        matrix = csr_matrix(tuple([parts['data'], parts['indices'], parts['indptr']]),
                            shape=parts['shape'])
        matrix_array = matrix.toarray()
    return matrix_array

In [ ]:
#import interval information, contains list of indices and chromosome that they correspond to

#argument:
# filename: input name of the file you wish to import intervals for

#output:
#interval_list: list of chromosomes, indices
#keychr: keys to access different list from interval_list

def get_intervals(filename):
    file = h5py.File(filename)
    key_list = file['intervals'].keys()
    interval_list = {}
    for key in key_list:
        interval_list[key] = file['intervals'][key][()]
    keychr = list(key_list)
    return interval_list, keychr

In [ ]:
#generate a matrix for specific chromosome vs chromosome interaction

#argument:
#ma: matrix of heatmap
#chromosome_x: chromosome to use for x axis
#chromosome_y: chromosome to use for y axis

#output:
#matrix of desired region

def call_region(ma,chromosome_x,chromosome_y):
    start_x = indices[chromosome_x]['start']
    end_x = indices[chromosome_x]['end']
    
    start_y = indices[chromosome_y]['start']
    end_y = indices[chromosome_y]['end']
    
    return  ma[start_x:end_x,start_y:end_y]

In [ ]:
# chunky code to get the end index for regions, requried for get_indices function
# argument:
# chrlist: full list of which chromosome each index correspond to

#output:
#end: end index for each chromosome

def get_end_idx(chrlist):
    start = {}
    end = {}
    i = 0
    for chromes in chrlist:
        if chromes == b'chrIV':
            end[chromes] = i
            i+=1
        elif chromes == b'chrXV':
            end[chromes] = i
            i+=1
        elif chromes == b'chrVII':
            end[chromes] = i
            i+=1
        elif chromes == b'chrXII':
            end[chromes] = i
            i+=1
        elif chromes == b'chrXVI':
            end[chromes] = i
            i+=1
        elif chromes == b'chrXIII':
            end[chromes] = i
            i+=1
        elif chromes == b'chrII':
            end[chromes] = i
            i+=1
        elif chromes == b'chrXIV':
            end[chromes] = i
            i+=1
        elif chromes == b'chrX':
            end[chromes] = i
            i+=1
        elif chromes == b'chrXI':
            end[chromes] = i
            i+=1
        elif chromes == b'chrV':
            end[chromes] = i
            i+=1
        elif chromes == b'chrVIII':
            end[chromes] = i
            i+=1
        elif chromes == b'chrIX':
            end[chromes] = i
            i+=1
        elif chromes == b'chrIII':
            end[chromes] = i
            i+=1
        elif chromes == b'chrVI':
            end[chromes] = i
            i+=1
        elif chromes == b'chrI':
            end[chromes] = i
            i+=1
        elif chromes == b'chrM':
            end[chromes] = i
            i+=1
        else:
            break
    return end

In [ ]:
# makes a dictionary of each chromosomes start and end index

# argument:
# chromosomes: the list of chromosomes
# end_idx: output of get_end_idx

#output:
#idxs: dictionary of each chromosomes start and end index

def get_indices(chromosomes,end_idx):
    idxs = {}
    next_start = 0
    for chromes in chromosomes:
        idxs[chromes] = {}
        idxs[chromes]['end'] = end_idx[chromes]+1

        if chromes == str(b'chrIV'):
            idxs[chromes]['start'] = 0
        else:
            idxs[chromes]['start'] = next_start
        next_start = end_idx[chromes]+1
        
    return idxs

In [ ]:
#divide 2 matrices

#argument: 
#nom_matrix: matrix on the nominator
#denom_matrix: matrix on the denominator
#log2: whether to apply log2 to output matrix

#output:
#matrix: matrix after division

def divide_matrices(nom_matrix, denom_matrix, log2 = True):
    divide_matrix = nom_matrix/denom_matrix

    if log2:
        matrix = np.log2(divide_matrix)
        matrix[~np.isfinite(matrix)] = 0
    else:
        matrix = divide_matrix
        matrix[~np.isfinite(matrix)] = 0
    return matrix

In [ ]:
#make the triangular matrix a full matrix by duplicating its diagonal counterpart

#argument:
#ma: matrix of heatmap

#output:
# full_matrix

def make_symmetric(ma):
    rotmatrix = np.transpose(ma)
    np.fill_diagonal(rotmatrix,0)
    full_matrix = ma + rotmatrix
    return full_matrix

In [ ]:
# import h5 files and produce the full matrix

#argument:
#filename: file name of the data to import

#output:
# full_matrix

def make_symm_matrix(filename):
    with tables.open_file(filename, 'r') as f:
        parts = {}
        try:
            for matrix_part in ('data', 'indices', 'indptr', 'shape'):
                parts[matrix_part] = getattr(f.root.matrix, matrix_part).read()
        except Exception as e:
            log.info('No h5 file. Please check parameters concerning the file type!')
            e
        matrix = csr_matrix(tuple([parts['data'], parts['indices'], parts['indptr']]),
                            shape=parts['shape'])
        matrix_array = matrix.toarray()
        
        full_ma = make_symmetric(matrix_array)

    return full_ma

In [ ]:
# 2d filter the matrix, eliminate high frequency noise

#argument:
#ma: matrix
#filter_length: length to apply the filter, larger length decreases max signal frequency

#output:
#filtered_ma: matrix after apply filtering

def filtering(ma, filter_length = 5):
    filtered_ma = signal.spline_filter(ma,lmbda=filter_length)
    return filtered_ma

In [ ]:
# plot the matrix as a heatmap

def plot_matrix(ma, filt = None,filter_len=0, symmetric = True, hlim = None,vlim =None, center=False, interpolation =None, cmap = 'RdYlBu_r', fig_size = 15,vcenter=None, vmin = None, vmax = None ):
    fig, ax = plt.subplots()

    ma[~np.isfinite(ma)] = 0
    if filt:
        filtered_ma = filtering(ma,filter_length = filter_len)
    else:
        filtered_ma = ma
    if symmetric:
        full_ma = filtered_ma
    else:
        full_ma = make_symmetric(filtered_ma)
    if center:
        norm = colors.TwoSlopeNorm(vmin=vmin, vcenter=vcenter, vmax=vmax)
        im = ax.imshow(full_ma,cmap = cmap,norm=norm)
    else:
        im = ax.imshow(full_ma,cmap = cmap,interpolation = interpolation,vmin = vmin, vmax = vmax)
    plt.ylim(vlim)
    plt.xlim(hlim)
    cb = fig.colorbar(im,fraction=0.046, pad=0.04)


In [ ]:
# use this to average only the nonzero values. To include also empty values, use np.average(ma)
# note this can also be done for 1d array
#argument:
#ma: matrix of interest

#output:
# averaged value of matrix
def finite_average(ma):
    return np.sum(ma)/np.count_nonzero(ma)

# Import data

In [ ]:
#get the interval and key information, choose which file to use with file_names

interval ,keychr = get_intervals(file_names[0])
print(interval)
print(keychr)

In [ ]:
# list of chromosomes

chromosomes = [b'chrIV',b'chrXV',b'chrVII',b'chrXII',b'chrXVI',b'chrXIII',b'chrII',b'chrXIV',b'chrX',b'chrXI',b'chrV',b'chrVIII',b'chrIX',b'chrIII',b'chrVI',b'chrI',b'chrM']

In [ ]:
# get start and end indices corresponding to each chromosomes

indices = get_indices(chromosomes,get_end_idx(interval[keychr[0]]))

In [ ]:
# get symmetric matrices from the files, naming nomenclature example: AA22 stands for Anchor Away 1st replicate of 2 hour time point

AA20 = make_symm_matrix(file_names[5])
AA22 = make_symm_matrix(file_names[3])
AA25 = make_symm_matrix(file_names[6])
AA30 = make_symm_matrix(file_names[4])
AA32 = make_symm_matrix(file_names[2])
AA35 = make_symm_matrix(file_names[7])
AA40 = make_symm_matrix(file_names[0])
AA42 = make_symm_matrix(file_names[1])
AA45 = make_symm_matrix(file_names[8])

------

## Averaging matrices

In [ ]:
# average heatmaps for 0 hour time point

AAave = np.zeros((2441, 2441))
for r in np.arange(0, 2441,1):
    for c in np.arange(0, 2441, 1):
        nom = np.array([ AA20[r, c], AA30[r, c], AA40[r,c]])
        denom = np.count_nonzero(nom) #count how many data points in nom are not zero
        if denom != 0:
            ave = np.sum(nom)/denom
            AAave[r, c] = ave
# turn anything not finite into 0 in the full matrix        
AAave[~np.isfinite(AAave)] = 0

In [ ]:
# average heatmaps for 2 hour time point

AA2ave = np.zeros((2441, 2441))
for r in np.arange(0, 2441,1):
    for c in np.arange(0, 2441, 1):
        nom = np.array([AA22[r, c], AA32[r, c], AA42[r,c]])
        denom = np.count_nonzero(nom) #count how many data points in nom are not zero
        if denom != 0:
            ave = np.sum(nom)/denom
            AA2ave[r, c] = ave
# turn anything not finite into 0 in the full matrix        
AA2ave[~np.isfinite(AA2ave)] = 0

In [ ]:
# average heatmaps for 5 hour time point

AA5ave = np.zeros((2441, 2441))
for r in np.arange(0, 2441,1):
    for c in np.arange(0, 2441, 1):
        nom = np.array([AA25[r, c], AA35[r,c], AA45[r,c]])
        denom = np.count_nonzero(nom) #count how many data points in nom are not zero
        if denom != 0:
            ave = np.sum(nom)/denom
            AA5ave[r, c] = ave
# turn anything not finite into 0 in the full matrix        
AA5ave[~np.isfinite(AA5ave)] = 0

In [ ]:
# ratio of 2 hour/DMSO and 5hour/DMSO

AAave2_0 = divide_matrices(AA2ave,AAave,log2=True)
AAave5_0 = divide_matrices(AA5ave,AAave,log2=True)

### Dictionary of genes

In [ ]:
ds2 = pd.read_csv('ds2h_idx.txt', delimiter='\t')

In [ ]:
ds2idx_array = ds2.iloc[:,5].values
ds2gs_array = (ds2.iloc[:,0].values)
ds2chr_array = ds2.iloc[:,4].values

In [ ]:
geneslist = ds2gs_array.tolist()
genes_idx = dict(zip(ds2gs_array, zip(ds2idx_array, ds2chr_array)))

In [ ]:
chromosomes = [4, 15, 7, 12, 16, 13, 2, 14, 10, 11, 5, 8, 9, 3, 6, 1]
chrstart = [0, 307, 526, 745, 961, 1151, 1336, 1499, 1656, 1806, 1940, 2056, 2169, 2257, 2321, 2376]

chrom_dict = dict(zip(chromosomes,chrstart))

In [ ]:
newidx = []
for gene in geneslist:
        chrom = genes_idx[gene][1]
        gene_idx = genes_idx[gene][0]
        for chroms in chromosomes:
            idx_start=chrom_dict[chroms]
            if chrom == chroms:
                newidx.append(idx_start+gene_idx)

gene_dict = dict(zip(geneslist,newidx))

## Multi-gene sampling

In [ ]:
TF = pd.read_csv('up5_ndd1_idx.txt', delimiter='\t')

In [ ]:
TFgs_array = (TF.iloc[:,1].values)
TFidx_array = (TF.iloc[:,4].values)
TFchr_array = (TF.iloc[:,3].values)

In [ ]:
TF_geneslist = TFgs_array.tolist()
TF_dict = dict(zip(TFgs_array, zip(TFidx_array, TFchr_array)))

In [ ]:
chromosomes = [4, 15, 7, 12, 16, 13, 2, 14, 10, 11, 5, 8, 9, 3, 6, 1]
chrstart = [0, 307, 526, 745, 961, 1151, 1336, 1499, 1656, 1806, 1940, 2056, 2169, 2257, 2321, 2376]

chrom_dict = dict(zip(chromosomes,chrstart))

In [ ]:
newidx = []
for gene in TF_geneslist:
        chrom = TF_dict[gene][1]
        gene_idx = TF_dict[gene][0]
        for chroms in chromosomes:
            idx_start=chrom_dict[chroms]
            if chrom == chroms:
                newidx.append(idx_start+gene_idx)

TF_newdict = dict(zip(TF_geneslist,newidx))

### TF genes average CF

In [ ]:
def make_heatmap(matrix,gene_dict):
    heat_map = []
    for key1 in list(gene_dict.keys()):
        idx1= gene_dict[key1]
        array = []
        for key2 in list(gene_dict.keys()):
            idx2 = gene_dict[key2]
            array.append(matrix[idx1, idx2])
        heat_map.append(array)
    return heat_map

In [ ]:
TF_5h = make_heatmap(AAave5_0, TF_newdict)

In [ ]:
x = map(np.array, TF_5h)

In [ ]:
x = np.array(list(x))

In [ ]:
x.shape

In [ ]:
finite_average(x)

### Null distribution

In [ ]:
from random import sample

In [ ]:
#Hundred thousand loops
length = len(TF_geneslist)
perm = 100000

def ave_cf():
    
    cf_list = []
    random_list = sample(geneslist,length)

    for gene in random_list:
        random_idx = gene_dict[gene]
        for TF_gene in TF_geneslist:
            TF_idx = TF_newdict[TF_gene]
        #change ma depending on 2h or 5h
        ma = AAave5_0
        cf = [ma[random_idx, TF_idx]]
        cf_list.append(cf)

    ave_cf = np.mean(cf_list)
    return ave_cf

ave_cf_rand = [ave_cf() for i in range(perm)]

In [ ]:
ave_cf_rand = np.array(ave_cf_rand)

plt.hist(ave_cf_rand, bins = 100, color='c')
plt.axvline(finite_average(ave_cf_rand), color='k', linestyle='dashed', linewidth=1)
plt.text(finite_average(ave_cf_rand),2500, np.around(finite_average(ave_cf_rand), decimals=2, out=None))
plt.axvline(finite_average(x), color='m', linewidth=1)
plt.text(finite_average(x),3000, np.around(finite_average(x), decimals=2, out=None), color='m')
plt.xlim(xmin=-0.6, xmax = 0.6)
plt.ylim(ymin=0, ymax = 4000)

tfmean = finite_average(x)
rmean = finite_average(ave_cf_rand)
rSD = np.std(ave_cf_rand)

randodist = np.histogram(ave_cf_rand, bins = 100)

freqlist = []

for i in enumerate(randodist[1]):
    if i[1] > tfmean:
        freq = randodist[0][i[0]-1]
        freqlist.append(freq)
        
pvfreq = np.sum(freqlist)
totalfreq = np.sum(randodist[0])
pvalue = pvfreq/totalfreq

plt.text(0.3,3300, 'p-value = {}'.format(np.around(pvalue, decimals=2, out=None)), color='k')

zscore = (finite_average(x)-finite_average(ave_cf_rand))/np.std(ave_cf_rand)
plt.text(0.3,3000, 'z-score = {}'.format(np.around(zscore, decimals=2, out=None)), color='k')

plt.xlabel('Average Change in Contact Frequency')
plt.ylabel('No. of Gene Sets')

##replace title and file name

plt.title('Ndd1 (Rap 5h Upregulated)')
plt.savefig('Up_5h_Ndd1.png')

